## Neural Network Models
____
GOALS:
* Model `price_actual` using a univariate XGBRegressor <br>
* Model all price components EXCEPT `price_day_ahead` with a multivariate XGBRegressor
___
OUTLINE:<br>
1. Import Libraries
2. Read in Data
3. Modeling `price_actual`<br>
    3.1 Prepare Data (Categoricals)<br>
    3.2 Split Data<br>
    3.3 Model one-to-one<br>
    3.4 Model 24-to-24<br>
    3.5 LSTM <br>
    3.6 LSTM-DNN <br>
4. Modeling Price Components (excluding `price_day_ahead`)<br>
    4.1 Model one-to-one<br>
    4.2 Model 24-to-24<br>
    4.3 LSTM <br>
    4.4 LSTM-DNN <br>
    
5. Final Model <br>
___

In [361]:
def window_gen(data, input_window, output_window, stride):
    '''
    Yields train and test samples of the given provided datasets, at specified input and out lengths, and specified strides
    
    PARAMETERS
    ----------
    train: tuple,
        Tuple of length 2, which provides the training features and training targets respectively
    test: tuple,
        Tuple of length 2, which provides the testing features and testing targets respectively
    input_window: int,
        Length of the sequence of input data
    output_window: int,
        Length of the sequence of output data
    stride, int
        Number of steps to move between first sample and second sample
    '''
    # Define X_train, y_train, X_test, y_test
    X, y = data[0], data[1]
    
    # Compute number of samples 
    n = len(X)/stride
    
    # If the input_window is greater than a day, X_train
    if input_window > 24:
        n_add = input_window - 24
        X = X.iloc[:n_add].append(X)
    else:
        n_add = 0
    for i in range(0, len(X)-n_add, stride):
        yield X.iloc[i:i+input_window].to_numpy(), y.iloc[i:i+output_window].to_numpy()
        
def resample(data, input_window, output_window, stride):
    win = window_gen((data[0], data[1]), input_window=input_window, output_window=output_window, stride=stride)
    
    n = int(len(data[0])/stride)
    X_data = np.array([])
    y_data = np.array([])
    
    for i in range(n):
        X_sample, y_sample = next(win)
        X_data = np.append(X_data, X_sample)
        y_data = np.append(y_data, y_sample)
        
    # Reshape
    X_data = X_data.reshape(n,input_window,len(data[0].columns))
    y_data = y_data.reshape(n, output_window)
    
    return X_data, y_data

In [362]:
def ensemble_nn(models):
    '''
    PARAMETERS
    ----------
    models: list,
        List containing trained models to use in ensemble
    RETURNS
    ----------
    ensemble: keras model,
        Trained model combining all input models into a single output model.
    '''
    # Get models in list
    models = [dnn, lstm]

    # Rename layers 
    for i, model in enumerate(models):
        for i2, layer in enumerate(model.layers):
            layer.trainable = False
            layer._name = f'ensemble_{i}_{i2}_{layer.name}'
    
    # Define multi-headed input
    ensemble_visible = [model.input for model in models]
    
    # Concatenate merge output from each model
    ensemble_outputs = [model.output for model in models]
    merge = layers.merge.concatenate(ensemble_outputs)
    hidden = layers.Dense(24, activation='relu')(merge)
    output = TimeDistributed(layers.Dense(1))(hidden)
    ensemble = keras.Model(inputs=ensemble_visible, outputs=output)   
    return ensemble

In [363]:
def compute_metrics(model, train, test):
    
    if type(train[0]) is not list:
        X_train = [train[0]]
        X_test = [test[0]]
    else:
        X_train = train[0]
        X_test = test[0]
    
    # Convert data into arrays if not already
    X_train = [np.array(x_data) for x_data in X_train]
    X_test = [np.array(x_data) for x_data in X_test]
    y_train, y_test = np.array(train[1]), np.array(test[1])
        
    # Convert data into arrays if not already
    #X_train, y_train = np.array(train[0]), np.array(train[1])
    #X_test, y_test = np.array(test[0]), np.array(test[1])
    
    # Predict 
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)
    
    # Compute sMAPE
    sMAPE_train = sMAPE(y_train.flatten(), preds_train.flatten())
    sMAPE_val = sMAPE(y_test.flatten(), preds_test.flatten())

    # Compute r2
    r2_train = r2(y_train.flatten(), preds_train.flatten())
    r2_val = r2(y_test.flatten(), preds_test.flatten())
    
    return [sMAPE_train, sMAPE_val, r2_train, r2_val]

### 1. Import Libraries

In [1]:
# Import statements
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from keras import layers, models, regularizers

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras.layers import TimeDistributed

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

import os
import winsound
from sklearn.preprocessing import LabelEncoder
import datetime as dt

os.chdir('../scripts')
from functions import impute_immediate_mean, split_data, SMAPE,sMAPE, r2, compute_metrics, to_supervised
os.chdir('../notebooks')


In [2]:
# Set up alarm for notification of model completion
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

### 2. Read in Data

In [3]:
df_lag =pd.read_csv('../data/clean/df_clean_lag.csv', index_col=0, parse_dates=True)
TSO_preds = df_lag.price_day_ahead.copy()
y_true_train = df_lag.loc[:'2019', 'price_actual'].copy()
y_true_val = df_lag.loc['2020', 'price_actual'].copy()

### 3. Modeling
### 3.1 Prepare Data
Scale Continuous Data

In [ ]:
continuous = df_lag.select_dtypes(exclude='object').drop(columns=['price_actual', 'price_day_ahead']).columns

# Get rid of negatives
time = dt.datetime(2021,3,24,22)
df_lag.loc[time, 'dew_point_bilbao_lag'] = impute_immediate_mean(df_lag['dew_point_bilbao_lag'], time)

# Rescale data [-1,1]
scaler = MinMaxScaler(feature_range=(-1, 1))
df_lag[continuous] = scaler.fit_transform(df_lag[continuous])

Encode and scale categoricals

In [4]:
# Get catergorical 
categorical = df_lag.select_dtypes(include='object').columns

# Get wind direction cols
wind_dirs = df_lag.filter(regex='wind(?!_speeds|_forecast)').columns

# Instantiate encoder and transfrom wind_dir cols
wind_dir_coder = LabelEncoder()
wind_dir_coder.fit(df_lag['wind_madrid_lag'])
for col in wind_dirs:
    df_lag[col] = wind_dir_coder.transform(df_lag[col])
    
# Stack condition columns into single col
stacked_conditions = df_lag.filter(regex='condition').stack()

# Instantiate Label encoder, fit and transform on condition cols
condition_coder = LabelEncoder()
condition_coder.fit(stacked_conditions)
for col in df_lag.filter(regex='condition').columns:
    df_lag[col] = condition_coder.transform(df_lag[col])

# Rescale data [-1,1]
df_lag[categorical] = scaler.fit_transform(df_lag[categorical])

Split Data into training and validation

In [6]:
# Get price components to drop
price_drop = df_lag.filter(regex='price_(?!actual|day)').columns

# Split data
X_train, y_train, X_val, y_val = split_data(df_lag.drop(columns=price_drop), 2020, 'price_actual')

Create results_actual dataframe

In [7]:
TSO_metrics = [round(sMAPE(y_true_train, TSO_preds.loc[:'2019']),3),
               round(sMAPE(y_true_val, TSO_preds.loc['2020']),3), 
               round(r2(y_true_train, TSO_preds.loc[:'2019']),3),
               round(r2(y_true_val, TSO_preds.loc['2020']),3)]
results_actual = pd.DataFrame({'TSO':TSO_metrics}, 
                              index=['sMAPE_train', 'sMAPE_val', 'r2_train', 'r2_val'])

### 3.3 Modeling (one-to-one)

In [360]:
# Define input_shape
input_shape = (X_train.shape[1],)

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(layers.Dense(1, activation='relu'))


In [11]:
nn1 = compile_fit(nn, (X_train,y_train), (X_val, y_val), patience=10,
                  loss = tf.keras.metrics.mean_absolute_error)

Epoch 1/200
1369/1369 [==============================] - 3s 2ms/step - loss: 2.0712 - SMAPE: 4.3372 - val_loss: 1.4228 - val_SMAPE: 4.1740
Epoch 2/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.4814 - SMAPE: 2.8341 - val_loss: 1.5818 - val_SMAPE: 4.6781
Epoch 3/200
1369/1369 [==============================] - 2s 2ms/step - loss: 1.4051 - SMAPE: 2.6793 - val_loss: 1.5677 - val_SMAPE: 4.7071
Epoch 4/200
1369/1369 [==============================] - 3s 2ms/step - loss: 1.3545 - SMAPE: 2.5828 - val_loss: 1.5839 - val_SMAPE: 4.8793
Epoch 5/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.3174 - SMAPE: 2.5079 - val_loss: 1.5754 - val_SMAPE: 4.8076
Epoch 6/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.2824 - SMAPE: 2.4439 - val_loss: 1.5899 - val_SMAPE: 4.8689
Epoch 7/200
1369/1369 [==============================] - 2s 2ms/step - loss: 1.2429 - SMAPE: 2.3634 - val_loss: 1.4458 - val_SMAPE: 4.4351
Epoch 8/200
1369/1369 [====

In [12]:
results_actual['nn1'] = compute_metrics(nn1, (X_train,y_train), (X_val, y_val))
results_actual

,TSO,nn1
sMAPE_train,16.030,2.910315
sMAPE_val,16.922,4.179681
r2_train,0.954,0.980623
r2_val,0.971,0.977432


### 3.5 Modeling (24-to-24)

Restructure data for 24 hour input and output

In [18]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_lag.drop(columns=price_drop), 2020, 'price_actual')

# Reorganize the training and testing data into batches
X_train, y_train = resample((X_train, y_train), 24, 24, 24)
X_val, y_val = resample((X_val,y_val), 24, 24, 24)

In [23]:
# Define input_shape
input_shape = (X_train.shape[1], X_train.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(TimeDistributed(layers.Dense(1)))

In [24]:
nn2 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

Epoch 1/200
58/58 [==============================] - 1s 8ms/step - loss: 12.2007 - SMAPE: 31.7609 - val_loss: 4.0983 - val_SMAPE: 10.5374
Epoch 2/200
58/58 [==============================] - 0s 6ms/step - loss: 2.1361 - SMAPE: 4.1268 - val_loss: 2.3717 - val_SMAPE: 6.8199
Epoch 3/200
58/58 [==============================] - 0s 5ms/step - loss: 1.7626 - SMAPE: 3.4348 - val_loss: 1.8982 - val_SMAPE: 5.7040
Epoch 4/200
58/58 [==============================] - 0s 5ms/step - loss: 1.7156 - SMAPE: 3.3673 - val_loss: 2.1122 - val_SMAPE: 6.4022
Epoch 5/200
58/58 [==============================] - 0s 5ms/step - loss: 1.6477 - SMAPE: 3.1796 - val_loss: 3.3190 - val_SMAPE: 9.3461
Epoch 6/200
58/58 [==============================] - 0s 5ms/step - loss: 1.6403 - SMAPE: 3.2754 - val_loss: 1.6228 - val_SMAPE: 4.8660
Epoch 7/200
58/58 [==============================] - 0s 5ms/step - loss: 1.7401 - SMAPE: 3.3245 - val_loss: 1.4932 - val_SMAPE: 4.3720
Epoch 8/200
58/58 [==============================] -

In [26]:
results_actual['nn2'] = compute_metrics(nn2, (X_train, y_train), (X_val, y_val))
results_actual

,TSO,nn1,nn2
sMAPE_train,16.030,2.910315,3.962906
sMAPE_val,16.922,4.179681,3.454599
r2_train,0.954,0.980623,0.983838
r2_val,0.971,0.977432,0.980697


### Model (LSTM)

In [27]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_lag.drop(columns=price_drop), 2020, 'price_actual')

# Reorganize the training and testing data into batches
X_train, y_train = resample((X_train, y_train), 24*7, 24, 24)
X_val, y_val = resample((X_val,y_val), 24*7, 24, 24)

In [28]:
input_shape = (X_train.shape[1], X_train.shape[2])
# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(60, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

In [29]:
nn3 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

Epoch 1/200
58/58 [==============================] - 7s 80ms/step - loss: 53.9754 - SMAPE: 179.1431 - val_loss: 35.6774 - val_SMAPE: 148.6349
Epoch 2/200
58/58 [==============================] - 4s 69ms/step - loss: 50.0152 - SMAPE: 154.3059 - val_loss: 33.4253 - val_SMAPE: 131.5672
Epoch 3/200
58/58 [==============================] - 4s 68ms/step - loss: 48.1394 - SMAPE: 143.7396 - val_loss: 31.7212 - val_SMAPE: 119.7844
Epoch 4/200
58/58 [==============================] - 4s 68ms/step - loss: 46.5005 - SMAPE: 135.2807 - val_loss: 30.1331 - val_SMAPE: 109.5757
Epoch 5/200
58/58 [==============================] - 4s 69ms/step - loss: 44.9406 - SMAPE: 127.1814 - val_loss: 28.5983 - val_SMAPE: 100.3414
Epoch 6/200
58/58 [==============================] - 4s 69ms/step - loss: 43.4204 - SMAPE: 120.0789 - val_loss: 27.0997 - val_SMAPE: 91.8631
Epoch 7/200
58/58 [==============================] - 4s 68ms/step - loss: 41.9306 - SMAPE: 113.1782 - val_loss: 25.6314 - val_SMAPE: 84.0301
Epoch 8/

In [30]:
results_actual['nn3'] = compute_metrics(nn3,(X_train,y_train), (X_val,y_val))
results_actual

,TSO,nn1,nn2,nn3
sMAPE_train,16.030,2.910315,3.962906,6.398275
sMAPE_val,16.922,4.179681,3.454599,7.789913
r2_train,0.954,0.980623,0.983838,0.869208
r2_val,0.971,0.977432,0.980697,0.853570


In [31]:
input_shape = (X_train.shape[1], X_train.shape[2])
# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(83, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

In [32]:
nn4 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

Epoch 1/200
58/58 [==============================] - 9s 115ms/step - loss: 53.0826 - SMAPE: 173.9812 - val_loss: 34.9559 - val_SMAPE: 142.9651
Epoch 2/200
58/58 [==============================] - 6s 109ms/step - loss: 49.5362 - SMAPE: 150.9076 - val_loss: 33.0619 - val_SMAPE: 128.9688
Epoch 3/200
58/58 [==============================] - 7s 114ms/step - loss: 47.8252 - SMAPE: 141.9558 - val_loss: 31.4508 - val_SMAPE: 117.9920
Epoch 4/200
58/58 [==============================] - 6s 109ms/step - loss: 46.2551 - SMAPE: 133.8312 - val_loss: 29.9100 - val_SMAPE: 108.1947
Epoch 5/200
58/58 [==============================] - 7s 120ms/step - loss: 44.7333 - SMAPE: 126.7064 - val_loss: 28.4060 - val_SMAPE: 99.2231
Epoch 6/200
58/58 [==============================] - 6s 112ms/step - loss: 43.2401 - SMAPE: 118.9267 - val_loss: 26.9321 - val_SMAPE: 90.9449
Epoch 7/200
58/58 [==============================] - 6s 111ms/step - loss: 41.7710 - SMAPE: 112.6276 - val_loss: 25.4815 - val_SMAPE: 83.2552
Ep

In [34]:
results_actual['nn4'] = compute_metrics(nn4,(X_train,y_train), (X_val,y_val))
results_actual

,TSO,nn1,nn2,nn3,nn4
sMAPE_train,16.030,2.910315,3.962906,6.398275,29.473468
sMAPE_val,16.922,4.179681,3.454599,7.789913,15.500838
r2_train,0.954,0.980623,0.983838,0.869208,0.234718
r2_val,0.971,0.977432,0.980697,0.853570,0.365850


### 3.6 Model (LSTM-DNN)

In [364]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_lag.drop(columns=price_drop), 2020, 'price_actual')

In [365]:
# Get the x cols for lstm network, lagged cols
X_train_lstm = X_train.filter(regex='lag')
X_train_dnn = X_train.drop(columns=X_train_lstm.columns)

# Get the x cols for dnn network, forecast cols
X_val_lstm = X_val.filter(regex='lag')
X_val_dnn = X_val.drop(columns=X_val_lstm.columns)

In [366]:
# Reorganize the training and testing data into batches
X_train_dnn, y_train_dnn = resample((X_train_dnn, y_train), 24, 24, 24)
X_val_dnn, y_val_dnn = resample((X_val_dnn, y_val), 24, 24, 24)

# LSTM
X_train_lstm, y_train_lstm = resample((X_train_lstm, y_train), 24, 24, 24)
X_val_lstm, y_val_lstm = resample((X_val_lstm, y_val), 24, 24, 24)



In [367]:
input_shape = (X_train_dnn.shape[1], X_train_dnn.shape[2])
# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(TimeDistributed(layers.Dense(1)))

In [368]:
dnn = compile_fit(nn, (X_train_dnn, y_train_dnn), (X_val_dnn, y_val_dnn))

Epoch 1/200
58/58 [==============================] - 1s 9ms/step - loss: 8.8587 - SMAPE: 22.0936 - val_loss: 1.9343 - val_SMAPE: 7.1471
Epoch 2/200
58/58 [==============================] - 0s 5ms/step - loss: 2.7592 - SMAPE: 6.2133 - val_loss: 2.1643 - val_SMAPE: 6.3035
Epoch 3/200
58/58 [==============================] - 0s 5ms/step - loss: 2.5873 - SMAPE: 5.3551 - val_loss: 3.7219 - val_SMAPE: 9.7954
Epoch 4/200
58/58 [==============================] - 0s 6ms/step - loss: 2.6200 - SMAPE: 5.1111 - val_loss: 1.8945 - val_SMAPE: 5.8856
Epoch 5/200
58/58 [==============================] - 0s 6ms/step - loss: 2.3450 - SMAPE: 4.5875 - val_loss: 2.3989 - val_SMAPE: 6.9441
Epoch 6/200
58/58 [==============================] - 0s 6ms/step - loss: 2.3966 - SMAPE: 4.6159 - val_loss: 2.5255 - val_SMAPE: 7.2377
Epoch 7/200
58/58 [==============================] - 0s 5ms/step - loss: 2.2005 - SMAPE: 4.2830 - val_loss: 1.8122 - val_SMAPE: 5.5049
Epoch 8/200
58/58 [==============================] - 0

In [369]:
compute_metrics(dnn, (X_train_dnn, y_train_dnn), (X_val_dnn, y_val_dnn))

[6.156025545421572, 5.529825922826223, 0.9587082867775397, 0.9592410961929292]

In [370]:
input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])
# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(83, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train_lstm.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

lstm = compile_fit(nn, (X_train_lstm, y_train_lstm), (X_val_lstm, y_val_lstm))

Epoch 1/200
58/58 [==============================] - 4s 32ms/step - loss: 52.4887 - SMAPE: 170.0182 - val_loss: 32.4569 - val_SMAPE: 137.9465
Epoch 2/200
58/58 [==============================] - 1s 23ms/step - loss: 49.1854 - SMAPE: 149.8244 - val_loss: 30.5765 - val_SMAPE: 124.0315
Epoch 3/200
58/58 [==============================] - 1s 23ms/step - loss: 47.4650 - SMAPE: 140.6329 - val_loss: 28.9659 - val_SMAPE: 113.1006
Epoch 4/200
58/58 [==============================] - 1s 24ms/step - loss: 45.8898 - SMAPE: 132.3641 - val_loss: 27.4319 - val_SMAPE: 103.4150
Epoch 5/200
58/58 [==============================] - 1s 24ms/step - loss: 44.3654 - SMAPE: 124.6539 - val_loss: 25.9401 - val_SMAPE: 94.6012
Epoch 6/200
58/58 [==============================] - 1s 23ms/step - loss: 42.8722 - SMAPE: 117.7993 - val_loss: 24.4839 - val_SMAPE: 86.5248
Epoch 7/200
58/58 [==============================] - 1s 23ms/step - loss: 41.4040 - SMAPE: 110.9791 - val_loss: 23.0612 - val_SMAPE: 79.0947
Epoch 8/2

In [374]:
nn5 = ensemble_nn([dnn, lstm])

In [387]:
ensemble = compile_fit(nn5, ([X_train_dnn, X_train_lstm], y_train_dnn), ([X_val_dnn, X_val_lstm], y_val_dnn))

Epoch 1/200
58/58 [==============================] - 3s 19ms/step - loss: 2.0467 - SMAPE: 3.9244 - val_loss: 2.8477 - val_SMAPE: 7.9937
Epoch 2/200
58/58 [==============================] - 1s 11ms/step - loss: 2.0353 - SMAPE: 3.9383 - val_loss: 2.6392 - val_SMAPE: 7.4816
Epoch 3/200
58/58 [==============================] - 1s 11ms/step - loss: 2.0987 - SMAPE: 4.0259 - val_loss: 3.1108 - val_SMAPE: 8.5925
Epoch 4/200
58/58 [==============================] - 1s 12ms/step - loss: 2.0396 - SMAPE: 3.9123 - val_loss: 2.8235 - val_SMAPE: 7.9280
Epoch 5/200
58/58 [==============================] - 1s 12ms/step - loss: 2.0362 - SMAPE: 3.9659 - val_loss: 2.8346 - val_SMAPE: 7.9490
Epoch 6/200
58/58 [==============================] - 1s 11ms/step - loss: 2.0499 - SMAPE: 3.9636 - val_loss: 2.7079 - val_SMAPE: 7.6538
Epoch 7/200
58/58 [==============================] - 1s 11ms/step - loss: 2.0358 - SMAPE: 3.9406 - val_loss: 2.4815 - val_SMAPE: 7.0973
Epoch 8/200
58/58 [=============================

In [388]:
compute_metrics(ensemble, ([X_train_dnn, X_train_lstm], y_train_dnn), ([X_val_dnn, X_val_lstm], y_val_dnn))

[4.092027495943533, 7.146394956377796, 0.9588267237215046, 0.9596074221172163]

REPEAT BUT WITH NEW DNN and LSTM as provided by the paper

In [389]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_lag.drop(columns=price_drop), 2020, 'price_actual')

# Get the x cols for lstm network, lagged cols
X_train_lstm = X_train.filter(regex='lag')
X_train_dnn = X_train.drop(columns=X_train_lstm.columns)

# Get the x cols for dnn network, forecast cols
X_val_lstm = X_val.filter(regex='lag')
X_val_dnn = X_val.drop(columns=X_val_lstm.columns)

# Reorganize the training and testing data into batches
X_train_dnn, y_train_dnn = resample((X_train_dnn, y_train), 24, 24, 24)
X_val_dnn, y_val_dnn = resample((X_val_dnn, y_val), 24, 24, 24)

# LSTM
X_train_lstm, y_train_lstm = resample((X_train_lstm, y_train), 24*7, 24, 24)
X_val_lstm, y_val_lstm = resample((X_val_lstm, y_val), 24*7, 24, 24)

In [392]:
# Define dnn input shape
input_shape = (X_train_dnn.shape[1], X_train_dnn.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(184, activation='relu'))
nn.add(TimeDistributed(layers.Dense(1)))

dnn = compile_fit(nn, (X_train_dnn, y_train_dnn), (X_val_dnn, y_val_dnn))

Epoch 1/200
58/58 [==============================] - 1s 5ms/step - loss: 12.1998 - SMAPE: 31.6851 - val_loss: 2.0293 - val_SMAPE: 7.7098
Epoch 2/200
58/58 [==============================] - 0s 3ms/step - loss: 2.8323 - SMAPE: 6.6092 - val_loss: 1.8841 - val_SMAPE: 6.4578
Epoch 3/200
58/58 [==============================] - 0s 3ms/step - loss: 2.6425 - SMAPE: 5.7300 - val_loss: 1.8081 - val_SMAPE: 5.5943
Epoch 4/200
58/58 [==============================] - 0s 3ms/step - loss: 2.4959 - SMAPE: 5.0317 - val_loss: 2.0479 - val_SMAPE: 6.3141
Epoch 5/200
58/58 [==============================] - 0s 3ms/step - loss: 2.4786 - SMAPE: 4.9195 - val_loss: 3.2881 - val_SMAPE: 9.0273
Epoch 6/200
58/58 [==============================] - 0s 3ms/step - loss: 2.4749 - SMAPE: 4.9809 - val_loss: 2.1763 - val_SMAPE: 6.4997
Epoch 7/200
58/58 [==============================] - 0s 3ms/step - loss: 2.2631 - SMAPE: 4.5334 - val_loss: 2.4584 - val_SMAPE: 7.1421
Epoch 8/200
58/58 [==============================] - 

In [393]:
# Define lstm input shape
input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(83, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train_lstm.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

lstm = compile_fit(nn, (X_train_lstm, y_train_lstm), (X_val_lstm, y_val_lstm))

Epoch 1/200
58/58 [==============================] - 7s 88ms/step - loss: 52.3485 - SMAPE: 169.1976 - val_loss: 32.6204 - val_SMAPE: 139.2095
Epoch 2/200
58/58 [==============================] - 5s 83ms/step - loss: 49.4839 - SMAPE: 151.6029 - val_loss: 30.9504 - val_SMAPE: 126.6914
Epoch 3/200
58/58 [==============================] - 5s 85ms/step - loss: 47.8517 - SMAPE: 142.4603 - val_loss: 29.3624 - val_SMAPE: 115.7127
Epoch 4/200
58/58 [==============================] - 6s 102ms/step - loss: 46.2972 - SMAPE: 134.0880 - val_loss: 27.8452 - val_SMAPE: 105.9593
Epoch 5/200
58/58 [==============================] - 6s 111ms/step - loss: 44.7869 - SMAPE: 126.8512 - val_loss: 26.3623 - val_SMAPE: 97.0376
Epoch 6/200
58/58 [==============================] - 7s 115ms/step - loss: 43.3027 - SMAPE: 119.7703 - val_loss: 24.9107 - val_SMAPE: 88.8404
Epoch 7/200
58/58 [==============================] - 6s 103ms/step - loss: 41.8404 - SMAPE: 112.9714 - val_loss: 23.4879 - val_SMAPE: 81.2774
Epoch

In [396]:
# Define and fit ensemble combining dnn and lstm
nn6 = ensemble_nn([dnn, lstm])
nn6 = compile_fit(nn6, ([X_train_dnn, X_train_lstm], y_train_dnn), ([X_val_dnn, X_val_lstm], y_val_dnn))
compute_metrics(nn6, ([X_train_dnn, X_train_lstm], y_train_dnn), ([X_val_dnn, X_val_lstm], y_val_dnn))

Epoch 1/200
58/58 [==============================] - 5s 49ms/step - loss: 48.8106 - SMAPE: 152.4004 - val_loss: 25.1914 - val_SMAPE: 95.4864
Epoch 2/200
58/58 [==============================] - 2s 41ms/step - loss: 25.9820 - SMAPE: 60.0860 - val_loss: 5.1566 - val_SMAPE: 13.1575
Epoch 3/200
58/58 [==============================] - 2s 41ms/step - loss: 4.5506 - SMAPE: 8.4486 - val_loss: 4.5524 - val_SMAPE: 12.5807
Epoch 4/200
58/58 [==============================] - 2s 41ms/step - loss: 2.5549 - SMAPE: 4.9189 - val_loss: 4.2453 - val_SMAPE: 11.4921
Epoch 5/200
58/58 [==============================] - 2s 41ms/step - loss: 2.3918 - SMAPE: 4.5476 - val_loss: 3.7181 - val_SMAPE: 10.0014
Epoch 6/200
58/58 [==============================] - 2s 42ms/step - loss: 2.2978 - SMAPE: 4.4350 - val_loss: 3.3594 - val_SMAPE: 9.0714
Epoch 7/200
58/58 [==============================] - 2s 42ms/step - loss: 2.2374 - SMAPE: 4.2825 - val_loss: 3.3860 - val_SMAPE: 9.1491
Epoch 8/200
58/58 [==================

[4.285381318820285, 7.017124841839628, 0.9548516932368862, 0.9639203045491993]

REPEAT lstm 14 days

In [397]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_lag.drop(columns=price_drop), 2020, 'price_actual')

# Get the x cols for lstm network, lagged cols
X_train_lstm = X_train.filter(regex='lag')
X_train_dnn = X_train.drop(columns=X_train_lstm.columns)

# Get the x cols for dnn network, forecast cols
X_val_lstm = X_val.filter(regex='lag')
X_val_dnn = X_val.drop(columns=X_val_lstm.columns)

# Reorganize the training and testing data into batches
X_train_dnn, y_train_dnn = resample((X_train_dnn, y_train), 24, 24, 24)
X_val_dnn, y_val_dnn = resample((X_val_dnn, y_val), 24, 24, 24)


# LSTM
X_train_lstm, y_train_lstm = resample((X_train_lstm, y_train), 24*14, 24, 24)
X_val_lstm, y_val_lstm = resample((X_val_lstm, y_val), 24*14, 24, 24)

In [398]:
input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])
# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(83, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train_lstm.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

lstm = compile_fit(nn, (X_train_lstm, y_train_lstm), (X_val_lstm, y_val_lstm))

Epoch 1/200
58/58 [==============================] - 16s 231ms/step - loss: 50.6805 - SMAPE: 159.3007 - val_loss: 30.1632 - val_SMAPE: 121.1758
Epoch 2/200
58/58 [==============================] - 14s 247ms/step - loss: 46.9471 - SMAPE: 137.8129 - val_loss: 28.4009 - val_SMAPE: 109.4670
Epoch 3/200
58/58 [==============================] - 18s 310ms/step - loss: 45.3050 - SMAPE: 128.6257 - val_loss: 26.8415 - val_SMAPE: 99.8647
Epoch 4/200
58/58 [==============================] - 17s 299ms/step - loss: 43.7657 - SMAPE: 121.8247 - val_loss: 25.3457 - val_SMAPE: 91.2498
Epoch 5/200
58/58 [==============================] - 14s 240ms/step - loss: 42.2706 - SMAPE: 114.7420 - val_loss: 23.8941 - val_SMAPE: 83.3942
Epoch 6/200
58/58 [==============================] - 15s 260ms/step - loss: 40.8053 - SMAPE: 108.3950 - val_loss: 22.4802 - val_SMAPE: 76.1848
Epoch 7/200
58/58 [==============================] - 14s 243ms/step - loss: 39.3615 - SMAPE: 102.4579 - val_loss: 21.1020 - val_SMAPE: 69.53

In [399]:
compute_metrics(lstm, (X_train_lstm, y_train_lstm), (X_val_lstm, y_val_lstm))

[38.70429814577969,
 24.199855281584426,
 0.006188302756964411,
 0.006700144850133799]

In [400]:
# Define and fit ensemble combining dnn and lstm
nn7 = ensemble_nn([dnn, lstm])
nn7 = compile_fit(nn7, ([X_train_dnn, X_train_lstm], y_train_dnn), ([X_val_dnn, X_val_lstm], y_val_dnn))
compute_metrics(nn7, ([X_train_dnn, X_train_lstm], y_train_dnn), ([X_val_dnn, X_val_lstm], y_val_dnn))

Epoch 1/200
58/58 [==============================] - 9s 106ms/step - loss: 33.9910 - SMAPE: 92.0138 - val_loss: 11.0794 - val_SMAPE: 33.5418
Epoch 2/200
58/58 [==============================] - 5s 84ms/step - loss: 5.0876 - SMAPE: 10.0422 - val_loss: 1.9969 - val_SMAPE: 5.7985
Epoch 3/200
58/58 [==============================] - 5s 79ms/step - loss: 2.1946 - SMAPE: 4.2587 - val_loss: 2.4026 - val_SMAPE: 6.8783
Epoch 4/200
58/58 [==============================] - 5s 83ms/step - loss: 2.1666 - SMAPE: 4.2156 - val_loss: 2.5428 - val_SMAPE: 7.2214
Epoch 5/200
58/58 [==============================] - 5s 79ms/step - loss: 2.1531 - SMAPE: 4.1695 - val_loss: 2.6565 - val_SMAPE: 7.5595
Epoch 6/200
58/58 [==============================] - 4s 77ms/step - loss: 2.1440 - SMAPE: 4.2788 - val_loss: 2.7295 - val_SMAPE: 7.7558
Epoch 7/200
58/58 [==============================] - 4s 75ms/step - loss: 2.1413 - SMAPE: 4.1259 - val_loss: 2.7389 - val_SMAPE: 7.7705
Epoch 8/200
58/58 [=======================

[4.421217783894029, 5.816508209817018, 0.9536169132453041, 0.9648005251925975]

ENCODER/DECODER LSTM

**ENCODER**:<br>
One or more LSTM layers can be used to implement the encoder model. The output is a fixed-size vector that represents the internal represntation of the input sequence. The number of memory cells in this layer defines the length of this fixed-size vector.
* Produces a two dimensional matrix of outputs, where the length is defined by the number of memory cells in the layer.

**DECODER**:
Must transform the learned interal representation of the input sequence into the correct output sequence. The output should be a TimeDistributed layer.
* Expects a 3D input of [samples, time steps, features] in order to produce a decoded sequence of some different length define by the problem

**Repeat Vector**:
* Repeats the provided 2D input multiple times to create a 3D output
* Use to transition from ENCODER - DECORDER


### Model CNN

In [408]:
X_train, y_train, X_val, y_val = split_data(df_lag.drop(columns = price_drop), 2020, 'price_actual')

# Reorganize the training and testing data into batches
X_train, y_train = resample((X_train, y_train), 24, 24, 24)
X_val, y_val = resample((X_val, y_val), 24, 24, 24)

In [411]:
X_train.shape

(1825, 24, 60)

In [410]:
# Define input_shape
input_shape = (X_train.shape[1], X_train.shape[2])

cnn = models.Sequential()
cnn.add(layers.Dense(60, activation='relu', input_shape=input_shape))
cnn.add(layers.ConvLSTM1D())

(366, 24, 60)

### 4. Modeling Proce Components
### 4.1 One-to-one

In [418]:
# Split data
X_train, y_train, X_val, y_val = split_data(df_lag.drop(columns=['price_actual', 'price_day_ahead']), 2020, list(price_drop))

In [423]:
y_train.shape

(43800, 14)

In [424]:
# Define input_shape
input_shape = (X_train.shape[1],)

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(X_train.shape[1], activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(layers.Dense(14, activation='relu'))

nn8 = compile_fit(nn, (X_train,y_train), (X_val, y_val), patience=10,
                  loss = tf.keras.metrics.mean_absolute_error)

Epoch 1/200
1369/1369 [==============================] - 3s 2ms/step - loss: 0.6098 - SMAPE: nan - val_loss: 0.6820 - val_SMAPE: nan
Epoch 2/200
1369/1369 [==============================] - 2s 2ms/step - loss: 0.6050 - SMAPE: nan - val_loss: 0.6824 - val_SMAPE: nan
Epoch 3/200
1369/1369 [==============================] - 2s 2ms/step - loss: 0.6035 - SMAPE: nan - val_loss: 0.6841 - val_SMAPE: nan
Epoch 4/200
1369/1369 [==============================] - 2s 2ms/step - loss: 0.6024 - SMAPE: nan - val_loss: 0.6822 - val_SMAPE: nan
Epoch 5/200
1369/1369 [==============================] - 2s 2ms/step - loss: 0.6017 - SMAPE: nan - val_loss: 0.6839 - val_SMAPE: nan
Epoch 6/200
1369/1369 [==============================] - 2s 2ms/step - loss: 0.6014 - SMAPE: nan - val_loss: 0.6835 - val_SMAPE: nan
Epoch 7/200
1369/1369 [==============================] - 2s 2ms/step - loss: 0.6010 - SMAPE: nan - val_loss: 0.6838 - val_SMAPE: nan
Epoch 8/200
1369/1369 [==============================] - 2s 2ms/step 

In [426]:
preds = nn8.predict(X_train)

In [427]:
preds.shape

(43800, 14)

In [11]:
nn1 = compile_fit(nn, (X_train,y_train), (X_val, y_val), patience=10,
                  loss = tf.keras.metrics.mean_absolute_error)

Epoch 1/200
1369/1369 [==============================] - 3s 2ms/step - loss: 2.0712 - SMAPE: 4.3372 - val_loss: 1.4228 - val_SMAPE: 4.1740
Epoch 2/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.4814 - SMAPE: 2.8341 - val_loss: 1.5818 - val_SMAPE: 4.6781
Epoch 3/200
1369/1369 [==============================] - 2s 2ms/step - loss: 1.4051 - SMAPE: 2.6793 - val_loss: 1.5677 - val_SMAPE: 4.7071
Epoch 4/200
1369/1369 [==============================] - 3s 2ms/step - loss: 1.3545 - SMAPE: 2.5828 - val_loss: 1.5839 - val_SMAPE: 4.8793
Epoch 5/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.3174 - SMAPE: 2.5079 - val_loss: 1.5754 - val_SMAPE: 4.8076
Epoch 6/200
1369/1369 [==============================] - 2s 1ms/step - loss: 1.2824 - SMAPE: 2.4439 - val_loss: 1.5899 - val_SMAPE: 4.8689
Epoch 7/200
1369/1369 [==============================] - 2s 2ms/step - loss: 1.2429 - SMAPE: 2.3634 - val_loss: 1.4458 - val_SMAPE: 4.4351
Epoch 8/200
1369/1369 [====